# Grabbing Frames from an SD Card

The most basic example, point us at an SD card and start grabbin!

In [ ]:
from pprint import pprint
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore")

from miniscope_io.io import SDCard
from miniscope_io.formats import WireFreeSDLayout

The `WireFreeSDConfig` object is an instance of the `sdcard.SDLayout` class. The `SDLayout`
class is a generic way of declaring how data is laid out on an SD card, and the WireFree layout
is one instance of that! This layer of abstraction lets us write reusable code for grabbing
frames, and also gives us a way of storing, communicating, and reasoning about SD card layouts
in a way that's a little more flexible than hardcoding constants. In a future version we
probably want to be more explicit about how to map individual sectors of the model to the
SD card so we can do stuff like random seek and whatnot, but this will do for now!

It looks like this:

In [ ]:
pprint(WireFreeSDLayout.dict(), sort_dicts=False)

Using the SDCard class with the SDLayout class is straightforward, but since I don't have a
data sample I can't actually test that it works right now!

In the future once I get an image we would want to make some jupyter widget to select possible drives,
but for now i'll just hardcode it as a string for the sake of an example. The `SDCard` class has a
`check_valid` method that looks for the `WRITE_KEY`s as in the prior notebook, so we could also
make that into a classmethod and just automatically find the right drive that way.

In [ ]:
#For reading out SD card image file
#drive = Path('..') / 'data' / 'test.img'
#directpath = False

#For directly reading out SD card. Drive number can be found with Get-PhysicalDisk command (for Windows Powershell)
drive = r"\\.\PhysicalDrive3"
directpath = True

In [ ]:
sd = SDCard(drive=drive, layout = WireFreeSDLayout, directpath=directpath)

pprint(sd.config.dict())

In [ ]:
fig, ax = plt.subplots()

# stashing the matplotlib frames to make an animation
frames = []

with sd:
    # Seek to 5 seconds in
    sd.frame = sd.config.fs * 5

    # Play for 5 seconds
    for i in range(sd.config.fs * 5):
        frame = sd.read()
        frames.append([ax.imshow(frame, cmap='gray', animated=True)])

# Show animation
anim = animation.ArtistAnimation(fig, frames, interval=sd.config.fs, blit=True)
display(HTML(anim.to_jshtml(fps=sd.config.fs)))
    